# Two random variables

This becomes much more interesting, but already a lot more complex.

Start with two loads. Illustrated using
- intuition
- Venn diagrams
- bivariate plot

Then we will consider:
- joint / and (discrete)
- union / or (discrete)
- Brief illustration: $Z = R - S$


<div class="alert alert-block alert-success"><b>CSS formatted boxes</b>
    
This is created using HTML. Copy everything you see between the <code>&ltdiv ...&gt</code> and <code>&lt/div&gt</code> tags. The <code>alert</code> class allows changing the box color from green to yellow or red by changing the word <code>success</code> in the first <code>div</code> tag to <code>warning</code> and <code>danger</code>, respectively. You can only use HTML formatting within the colored box, Markdown does not work inside an HTML tag (this is also true in Jupyter notebooks).
</div>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as st
import pandas as pd

In [ ]:
from scipy.stats import multivariate_normal as mvn
import sympy as sym

In [ ]:
def norm_copula(u: float, v: float, rho: float) -> float:
    means = [0, 0]
    cov_matrix = [[1, rho], [rho, 1]]
    c = mvn(means, cov_matrix)
    return c.cdf([st.norm.ppf(u), st.norm.ppf(v)])

In [ ]:
norm_copula(1, 0.5, 0.8)

In [ ]:
def p_and(u: float, v: float, rho=0):
    c = norm_copula(u, v, rho)
    return 1-u-v+c

def p_or(u, v, rho):
    c = norm_copula(u, v, rho)
    return 1 - c

def p_cond1(u, v, rho):
    c = norm_copula(u, v, rho)
    return (1-u)*(1-u-v+c)

def p_cond2(u, v, rho):
    c = norm_copula(u, v, rho)
    return 1 - c/u

def p_cond3(u, v, rho):
    ''' Dirty way: only works for bivariate normal '''
    x = st.norm.ppf(u)
    y = st.norm.ppf(v)
    mu_x = rho*y
    sigma_x = np.sqrt(1-rho**2)
    return 1-st.norm.cdf(x, mu_x, sigma_x)
    
def p_K(t, rho, N=1000):
    u = np.random.uniform(0, 1, size=N)
    v = np.random.uniform(0, 1, size=N) 
    k = 0
    for i in range(N):
        if norm_copula(u[i], v[i], rho)>t:
            k += 1
    return k/N

In [ ]:
p_K(0.7, 0)

In [ ]:
p_and(0.5, 0.5, 0.5)

In [ ]:
def plot_figures(u, v, rho):
    f, ax = plt.subplots(6, figsize=(13,10))
    # Figure p_AND
    ax[0].vlines(u, ymin=0, ymax=v, colors='k', linestyles="dashed")
    ax[0].hlines(v, xmin=0, xmax=u, colors='k', linestyles="dashed")
    ax[0].fill_between([u, 1], v, 1, color="grey", linewidth=2, edgecolor="k", alpha=1)
    ax[0].axvspan(0,1,0,1, edgecolor="k", linewidth=4, facecolor="None")
    ax[0].set_title("$p_{AND}$")
    
    #Figure p_OR
    ax[1].vlines(u, ymin=v, ymax=1, colors='k', linestyles="dashed")
    ax[1].hlines(v, xmin=u, xmax=1, colors='k', linestyles="dashed")
    ax[1].fill_between(np.array([0, u, u, 1]), np.array([v, v, 0, 0]), 1, color="grey", linewidth=2, edgecolor="k", alpha=1) 
    ax[1].axvspan(0,1,0,1, edgecolor="k", linewidth=4, facecolor="None")
    ax[1].set_title("$p_{OR}$")
    
    #Figure p_cond1
    ax[2].vlines([0, u, 1], ymin=0, ymax=1, colors='k', linestyles="dashed")
    ax[2].hlines([0, v, 1], xmin=0, xmax=u, colors='k', linestyles="dashed")
    ax[2].fill_between([u, 1], v, 1, color="grey", linewidth=2, edgecolor="k", alpha=1)
    ax[2].axvspan(u,1,0,1, edgecolor="k", linewidth=4, facecolor="None")
    ax[2].set_title("$p_{COND1}$")
    
    # Figure p_cond2
    ax[3].vlines([0, u, 1], ymin=0, ymax=1, colors='k', linestyles="dashed")
    ax[3].hlines([0, v, 1], xmin=0, xmax=1, colors='k', linestyles="dashed")
    ax[3].fill_between([0, u], v, 1, color="grey", linewidth=2, edgecolor="k", alpha=1)
    ax[3].axvspan(0,u,0,1, edgecolor="k", linewidth=4, facecolor="None")
    ax[3].set_title("$p_{COND2}$")
    
    # Figure p_cond3
    ax[4].vlines([0, u, 1], ymin=0, ymax=1, colors='k', linestyles="dashed")
    ax[4].hlines([0, v, 1], xmin=0, xmax=1, colors='k', linestyles="dashed")
    ax[4].vlines([u, u], ymin=[0, v], ymax=[v, 1], colors=["black", "grey"], linewidth=3, alpha=1)
    ax[4].set_title("$p_{COND3}$")
    
    for i in range(6):
        ax[i].axis("scaled")
    plt.setp(ax, xlim=(0,1), ylim=(0,1))
    plt.tight_layout()
    plt.show()
    return None

In [ ]:
plot_figures(0.75, 0.2, 0.7)